# Advent of Code 2021

In [1]:
using Pipe
using BenchmarkTools
using DelimitedFiles

### Day 1

https://adventofcode.com/2021/day/1

1. Count the number of times a depth measurement increases from the previous measurement.

2. Count the number of times the sum of measurements in this sliding window increases from the previous sum.

In [2]:
input = parse.(Int, eachline("inp/input01"))

part1(input) = count(diff(input) .> 0)
part2(input) = count(i -> sum(input[i+1:i+3]) > sum(input[i:i+2]), 1:length(input)-3)

@btime part1(input), part2(input) 

  94.900 μs (4001 allocations: 332.39 KiB)


(1713, 1734)

### Day 2

https://adventofcode.com/2021/day/2

1. Calculate the horizontal position and depth you would have after following the planned course.

2. Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course.

In [3]:
moves = readdlm("inp/input02")

function part1(moves)   
    horisontal = 0; depth = 0
    for move in eachrow(moves)
        if move[1] == "down"
            depth += move[2]
        elseif move[1] == "up"; 
            depth -= move[2]
        elseif move[1] == "forward"
            horisontal += move[2]
        end
    end
    horisontal * depth
end

function part2(moves)
    aim = 0; horisontal = 0; depth = 0
    for move in eachrow(moves)
        if move[1] == "down"
            aim += move[2]
        elseif move[1] == "up"
            aim -= move[2]
        elseif move[1] == "forward"
            horisontal += move[2]
            depth += move[2] * aim
        end
    end
    horisontal * depth
end

@btime part1(moves), part2(moves)

  117.200 μs (1906 allocations: 29.80 KiB)


(2322630, 2105273490)

### Day 3

https://adventofcode.com/2021/day/3

1. Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. What is the power consumption of the submarine?

2. Use the binary numbers in your diagnostic report to calculate the oxygen generator rating and CO2 scrubber rating, then multiply them together. What is the life support rating of the submarine??

In [4]:
report = @pipe readlines("inp/input03") .|> collect .|> parse.(Int, _) |> 
    hcat(_...) |> permutedims

to_decimal(v) = @pipe v |> join |> parse(Int, _, base = 2)
flip(i) = i == 1 ? 0 : 1
find_most_common(v) = @pipe median(v, dims=1) |> ceil.(Int, _)

function part1(report)
    γ = find_most_common(report)
    ε = flip.(γ)

    (γ, ε) .|> to_decimal |> prod
end
    
function part2(report)
    function f(r, i, use_least_common)
        size(r, 1) == 1 && return r
        val = find_most_common(r[:, i])
        if use_least_common; val = flip(val); end
        # select all the rows that have the most / least common number, 
        # and advance the recursion by one column
        f(r[r[:, i] .== val, :], i +1, use_least_common)
    end

    γ = f(report, 1, false)
    ε = f(report, 1, true)

    (γ, ε) .|> to_decimal |> prod
end
    
@btime part1(report), part2(report)

  201.600 μs (1336 allocations: 377.75 KiB)


(3309596, 11524610)

### Day 4

https://adventofcode.com/2021/day/4

1. To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?

2. Figure out which board will win last. Once it wins, what would its final score be?

In [5]:
function read_input(input)
    input = readlines(input)
    numbers = @pipe input[1] |> split(_, ",") |> parse.(Int, _)

    boards = []; line = 2
    while line <= length(input)
        @pipe [input[line + i] for i in 1:5] .|> 
            split |> hcat(_...) |> permutedims |> # changes data to 2-dimensional array
            parse.(Int, _) |>
            push!(boards, _)
        line += 6
    end
    
    boards, numbers
end
    
check_if_win(m) = any(any(all(i -> i == false, m, dims = dim)) for dim in 1:2)

function get_win_sum(numbers, board)
    # The unmarked array keeps track of which spaces on the boards are unmarked 
    # If a number is placed on tile on a board, set the unmarked value to 
    # false on the corresponding unmarked array and tile
    unmarked = trues(size(board))
    for (i, n) in enumerate(numbers)
        unmarked = (board .!= n) .* unmarked
        check_if_win(unmarked) && return(i, sum(board .* unmarked) * n)
    end
end

boards, numbers = read_input("inp/input04")

@btime @pipe boards .|> get_win_sum(numbers, _) |>
    sort(_)[[1, end]] |>
    getindex.(_, 2)

  2.836 ms (60641 allocations: 2.44 MiB)


2-element Vector{Int64}:
 11774
  4495

### Day 5

https://adventofcode.com/2021/day/5

1. Consider only horizontal and vertical lines. At how many points do at least two lines overlap?

2. Consider all of the lines. At how many points do at least two lines overlap?

In [6]:
lines = @pipe readlines("inp/input05") |>
    split.(_, r",| -> ") |>
    # add one to each number to accomodate Julia's 1 based arrays
    map.(line -> parse.(Int, line) +1, _)

# the size of the diagram the lines are drawn on
dim = maximum(hcat(lines...))
    
# step depends on whether you draw from lower values to higher, or from higher values to lower
get_range(c) = range(c[1], c[2], step = c[1] < c[2] ? 1 : -1)

function count_overlaps(lines, dim, use_diagonals)
    diagram = zeros(Int16, dim, dim)
    for line in lines
        range_x = get_range(line[[1,3]])
        range_y = get_range(line[[2,4]])
            
        if (length(range_x) == 1 || length(range_y) == 1) # draw straight lines
            diagram[range_x, range_y] .+= 1
        elseif use_diagonals # draw diagonal lines
            diagram[[CartesianIndex(elem) for elem in zip(range_x, range_y)]] .+= 1
        end
    end
    count(diagram .> 1)
end

@btime count_overlaps(lines, dim, false), count_overlaps(lines, dim, true)

  2.583 ms (5014 allocations: 6.20 MiB)


(5197, 18605)

### Day 6

https://adventofcode.com/2021/day/6

1. How many lanternfish would there be after 80 days?

2. How many lanternfish would there be after 256 days?

In [7]:
using OffsetArrays
using StatsBase: counts
O = OffsetArray

🐟 = @pipe readline("inp/input06") |> split(_, ',') |> parse.(Int, _) |>
    counts(_, 0:8) |>
    O(_, 0:8)

function count_fish(🐟, days)
    for day in 1:days
        🐟 = O([🐟[1:6];  🐟[7] + 🐟[0]; 🐟[8]; 🐟[0]], 0:8)
    end
    sum(🐟)
end

@btime count_fish(🐟, 80), count_fish(🐟, 256)

  1.251 ms (27973 allocations: 678.62 KiB)


(387413, 1738377086345)

### Day 7

https://adventofcode.com/2021/day/7

1. Determine the horizontal position that the crabs can align to using the least fuel possible. How much fuel must they spend to align to that position?

2. With updated fuel calculations

In [8]:
input = @pipe readline("inp/input07") |> split(_, ',') |> parse.(Int, _)
    
part1(input) = @pipe input .- median(input) .|> abs |> sum |> trunc(Int, _)

cost(position) = @pipe input .|> _-position .|> abs .|> binomial(_+1, 2) |> sum
part2(input) = minimum(cost, 1:length(input))

@btime part1(input), part2(input)

  17.150 ms (17006 allocations: 8.18 MiB)


(349812, 99763899)

### Day 8

https://adventofcode.com/2021/day/8

1. In the output values, how many times do digits 1, 4, 7, or 8 appear?

2. For each entry, determine all of the wire/segment connections and decode the four-digit output values. What do you get if you add up all of the output values?

In [9]:
input = split.(readlines("inp/input08"), " | ")
pattern_rows = getindex.(input, 1) .|> split
output_values = getindex.(input, 2) .|> split

function get_entry_value(patterns, output_values)

    # A dictionary with length of pattern as key, and the patterns with that length as values
    d = Dict(i => [] for i in 1:8)
    for p in patterns
        push!(d[length(p)], Set(p))
    end

    find_fives = function(s, d)
        if d[2][1] ⊆ s; 3
        elseif length(intersect(s, d[4][1])) == 2; 2
        else; 5; end
    end

    find_sixes = function(s, d)
        if d[4][1] ⊆ s; 9
        elseif d[2][1] ⊆ s; 0
        else; 6; end
    end

    dd = Dict(
        # These respond to a unique number of segments, so they each encode a unique digit
        d[2][1] => 1,
        d[3][1] => 7,
        d[4][1] => 4,
        d[7][1] => 8,
        # These have 3 with the same number of segments, so we have to use additional deduction
        d[5][1] => find_fives(d[5][1], d),
        d[5][2] => find_fives(d[5][2], d),
        d[5][3] => find_fives(d[5][3], d),
        d[6][1] => find_sixes(d[6][1], d),
        d[6][2] => find_sixes(d[6][2], d),
        d[6][3] => find_sixes(d[6][3], d))

    @pipe [dd[Set(elem)] for elem in output_values] |>
        parse(Int, join(_))
end

part1(output_values) = @pipe output_values |> 
    (length(elem) for row in _ for elem in row) |>
    count(in((2,3,4,7)), _)

part2(pattern_rows, output_values) = @pipe zip(pattern_rows, output_values) .|>  
    get_entry_value(_...) |>
    sum

@btime part1(output_values), part2(pattern_rows, output_values)

  1.501 ms (26604 allocations: 1.69 MiB)


(288, 940724)

### Day 9

https://adventofcode.com/2021/day/9

1. What is the sum of the risk levels of all low points on your heightmap?

2. What do you get if you multiply together the sizes of the three largest basins?

In [10]:
read_matrix(input) = @pipe readlines(input) .|> split(_, "") .|> 
    parse.(Int, _) |> hcat(_...) |> permutedims

moves = CartesianIndex.([(1,0), (0,1), (-1, 0), (0, -1)])
adjacent(p, board) = @pipe moves .|> p + _ |> filter(in(board), _)
larger_than_neighbours(p, M, board) = all(adj -> M[adj] > M[p], adjacent(p, board))
minimum_points(M, board) = filter(p -> larger_than_neighbours(p, M, board), board)

M = read_matrix("inp/input09")
board = CartesianIndices(M)

part1(M, board) = sum(p -> M[p] + 1, minimum_points(M, board))

function part2(M, board)
    
    function get_basin(p, M)    
        basin = Set([p])
        for adj in adjacent(p, M)
            if M[adj] > M[p] && M[adj] < 9
                basin = basin ∪ get_basin(adj, M)
            end
        end
        basin
    end

    @pipe (get_basin(min, M) for min in minimum_points(M, board)) .|>
        length |> sort |> _[end-2:end] |> prod
end

@btime part1(M, board), part2(M, board)

  17.276 ms (163337 allocations: 8.82 MiB)


(539, 1)

### Day 10

https://adventofcode.com/2021/day/10

1. Find the first illegal character in each corrupted line of the navigation subsystem. What is the total syntax error score for those errors?

2. Find the completion string for each incomplete line, score the completion strings, and sort the scores. What is the middle score?

In [22]:
using DataStructures: Stack
using Multibreak

input = readlines("inp/input10")

corrupted_scoring = Dict(')' => 3, ']' => 57,'}' => 1197, '>' => 25137)
completion_scoring = Dict('(' => 1, '[' => 2,'{' => 3, '<' => 4)
pairs = Dict(')' => '(', ']' => '[','}' => '{', '>' => '<')
openers = values(pairs)

function score_completion(stack)
    score = 0
    for elem in stack
        score = score * 5 + completion_scoring[elem]
    end
    score
end

function part1_2(input)
    corrupted_score = 0; completion_scores = []
    @multibreak for line in input
        stack = Stack{Char}()
        for char in line
            if char in openers
                push!(stack, char)
            else
                if pairs[char] != pop!(stack)
                    corrupted_score += corrupted_scoring[char]
                    break; continue
                end
            end
        end
        push!(completion_scores, score_completion(stack))
    end
    corrupted_score, round(Int, median(completion_scores))
end

@btime part1_2(input)

  388.600 μs (1287 allocations: 399.27 KiB)


(265527, 3969823589)

### Day 11

https://adventofcode.com/2021/day/11

1. How many total flashes are there after 100 steps?

2. What is the first step during which all octopuses flash?

In [12]:
CI = CartesianIndex; It = Iterators
read_matrix(input) = mapreduce(l -> parse.(Int, collect(l))', vcat, eachline(input))

adj = setdiff(CI(-1,-1):CI(1,1), (CI(0,0), ))

function step!(M)
    cavern = CartesianIndices(M)
    M .+= 1
    to_flash = Set(cavern[M .> 9])
    flashed = Set{CI{2}}()
    while !isempty(to_flash)
        flash = pop!(to_flash)
        push!(flashed, flash)
        for 🐙 in [flash + a for a in adj if flash + a ∈ cavern]
            M[🐙] += 1
            M[🐙] > 9 && 🐙 ∉ flashed && push!(to_flash, 🐙)
        end
    end
    M[collect(flashed)] .= 0
    length(flashed)
end

function part1(input)
    M = read_matrix(input)
    sum(step!(M) for _ in 1:100)
end

function part2(input)
    M = read_matrix(input)
    maximum(It.takewhile(_ -> step!(M) != 100, It.countfrom(1))) +1
end

@btime part1("inp/input11"), part2("inp/input11")

  13.313 ms (175071 allocations: 8.09 MiB)


(1669, 351)

### Day 13

https://adventofcode.com/2021/day/13

1. How many dots are visible after completing just the first fold instruction on your transparent paper?

2. Finish folding the transparent paper according to the instructions. The manual says the code is always eight capital letters. What code do you use to activate the infrared thermal imaging camera system?

In [23]:
using SparseArrays

function read_data(fn)
    dots, instructions = split.(split(read(fn, String), "\n\r\n"), "\r\n")
    instructions = @pipe instructions .|> (axis = _[12], line = parse(Int, _[14:end]) +1)
    dots = [parse.(Int, x) .+ 1 for x in split.(dots, ",")]
    sheet = sparse(last.(dots), first.(dots), true)
    sheet, instructions
end

sheet, instructions = read_data("inp/input13")

fold(sheet, i) = i.axis == 'y' ?
    sheet[1:i.line-1, :] .| reverse(sheet[i.line+1:end, :], dims = 1) :
    sheet[:, 1:i.line-1] .| reverse(sheet[:, i.line+1:end], dims = 2)

function solve(sheet, instructions)
    for instruction in instructions
        sheet = fold(sheet, instruction)
    end
    sheet
end

solve(sheet, [instructions[1]]) |> count |> println
solve(sheet, instructions)

942


6×40 SparseMatrixCSC{Bool, Int64} with 101 stored entries:
⠀⢹⠈⡩⠃⡎⣑⢸⠀⡇⣎⣱⢸⣉⠆⣏⡱⢸⠭⡂
⠑⠊⠘⠒⠂⠑⠚⠈⠒⠁⠃⠘⠘⠀⠀⠃⠑⠘⠒⠁

### Day 14

https://adventofcode.com/2021/day/14

1. How many dots are visible after completing just the first fold instruction on your transparent paper?

2. Finish folding the transparent paper according to the instructions. The manual says the code is always eight capital letters. What code do you use to activate the infrared thermal imaging camera system?

In [27]:
using DataStructures: DefaultDict
using StatsBase: countmap

function read_input(fn)
    input = readlines(fn)
    # character pairs as key, and counts as values
    polymer = @pipe input[1] |> countmap(_[i:i+1] for i in 1:length(_)-1)
    # character pairs as key, and inserted character as value
    rules = @pipe input[3:end] .|> split(_, " -> ") |> Dict(k=>v for (k,v) in _)
    last_char = input[1][end]
    polymer, rules, last_char
end

function step(polymer, rules)
    new_polymer = DefaultDict(0)
    # For each pair in polymer (AB), find the character x that
    # is inserted between them according to the rules (rules[pair]).
    # This creates two new pairs (Ax and xB), which are counted
    for (pair, n) in polymer
        new_polymer[pair[1] * rules[pair]] += n
        new_polymer[rules[pair] * pair[2]] += n
    end
    new_polymer
end

function score_polymer(polymer, last_char)
    counts = DefaultDict(0)
    for (pair, n) in polymer
        counts[pair[1]] += n
    end
    # If the very last char is part of a pair, that one is not counted above.
    counts[last_char] += 1
    min, max = values(counts) |> extrema
    max - min
end

function solve(polymer, rules, last_char, steps)
    for _ in 1:steps
        polymer = step(polymer, rules)
    end
    score_polymer(polymer, last_char)
end

polymer, rules, last_char = read_input("inp/input14")

@btime solve(polymer, rules, last_char, 10), solve(polymer, rules, last_char, 40)

  2.654 ms (67511 allocations: 2.43 MiB)


(5656, 12271437788530)

### Day 15

https://adventofcode.com/2021/day/15

1. What is the lowest total risk of any path from the top left to the bottom right?

2. Using the full map, what is the lowest total risk of any path from the top left to the bottom right?

In [28]:
# Solution 1. Dynamic programming. Works if you could only go down and right.

M = read_matrix("inp/input15_test")

function enlargen(M)
    M = [M    M.+1 M.+2 M.+3 M.+4
         M.+1 M.+2 M.+3 M.+4 M.+5
         M.+2 M.+3 M.+4 M.+5 M.+6
         M.+3 M.+4 M.+5 M.+6 M.+7
         M.+4 M.+5 M.+6 M.+7 M.+8] 
    M .= mod1.(M, 9)
end

function find_lowest_risk(M)
    N = deepcopy(M)
    dim = size(M, 1)
    for diag in (2*dim-1):-1:1
        for i in max(1, diag-dim):min(diag -1, dim)
            j = diag - i
            if i == dim
                low = N[i, j+1]
            elseif j == dim
                low = N[i+1, j]
            else
                low = min(N[i+1, j], N[i, j+1])
            end
            N[i, j] += low
        end
    end       
    min(N[1,2], N[2,1])
end

@btime enlargen(M) |> find_lowest_risk

  17.800 μs (30 allocations: 60.55 KiB)


315

###### Day 17

https://adventofcode.com/2021/day/17

1. What is the highest y position it reaches on this trajectory?

2. How many distinct initial velocity values cause the probe to be within the target area after any step?

In [29]:
#If x is negative, or y is positive, inverse the values.
target = (xmin = 192, xmax = 251, ymin = -89, ymax = -59)

function will_hit_target(velocity_x, velocity_y, target)
    x = 0; y = 0
    while y > target.ymin
        x += velocity_x
        y += velocity_y
        target.xmin <= x <= target.xmax && target.ymin <= y <= target.ymax && return true
        velocity_x = max(0, velocity_x - 1)
        velocity_y -= 1
    end
    false
end

# You can always hit the x-range. The highest you can go, is so that when you fall, you hit y.
# Your speed at 0 is equal to your initial y velocity. So you hit y-range if your initial
# y-velocity is lower in magnitude than the largest magnitude y value.
# (Note that if 0 is in y-range, the answer is infinite)
@btime (target.ymin^2 + target.ymin) ÷ 2,

# We know from part one that the y velocity is bounded by the magnitude of the largest y value.
# x of course is bounded by the largest x-range value.
count(will_hit_target(velocity_x, velocity_y, target) for 
        velocity_x in 1:target.xmax for velocity_y in target.ymin:-target.ymin)

  6.821 ms (137057 allocations: 6.24 MiB)


(3916, 2986)